In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from mlxtend.plotting import plot_decision_regions
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings("ignore")
#%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

In [ ]:
df.head()

In [ ]:
print("dimension of diabetes data: {}".format(df.shape)) #data boyutunu yazdık 

In [ ]:
print(df.groupby("Outcome").size()) #Outcome 1-0 dağılımı 

In [ ]:
sns.countplot(df["Outcome"],label = "Count") #Outcome görselleştirmesi yaptık
plt.show()

In [ ]:
df.info()


In [ ]:
#Veri analizi - pandas profiling
import pandas_profiling as pp
profile = df.profile_report(title='Şeker Hastalığı Raporu', pool_size = 0)
profile.to_file(output_file="output.html")

In [ ]:
df.columns

In [ ]:
print(df["Pregnancies"].value_counts(dropna = True))

In [ ]:
df.describe() #verinin istatistiksel bilgilerini aldık.

In [ ]:
df.boxplot(column = "Insulin", by = "Outcome")
plt.show()

In [ ]:
df_new = df.head()
df_new

In [ ]:
melted = pd.melt(frame=df_new, id_vars = 'Glucose', value_vars= ['Outcome','Pregnancies'])
melted

In [ ]:
melted.pivot(index = 'Glucose', columns = "variable", values = "value") #datayı melt ettik

In [ ]:
df1 = df.head()
df2 = df.tail()
conc_df = pd.concat([df1,df2], axis = 0, ignore_index = True) # axis = 0 : dataların satırlarını birleştirdik
conc_df

In [ ]:
df1 = df['Outcome'].head()
df2= df['Glucose'].head()
conc_df = pd.concat([df1,df2],axis =1) # axis = 1 : dataların sütunlarını birleştirdik
conc_df

In [ ]:
df.dtypes #data tiplerini görüyoruz

In [ ]:
df["BMI"] = df["BMI"].astype("int64") #data tipini değiştirdik float --> int
df["BMI"]

In [ ]:
df.info()

In [ ]:
df["Pregnancies"].value_counts(dropna =False) #kayıp veri var mı testi yaptık 

In [ ]:
df.columns[df.isnull().any()] #kayıp veri var-yok (varsa listeler)

In [ ]:
df.isnull().sum() #kayıp veri sayıları 

In [ ]:
df[df["Age"].isnull()] #kayıp veri varsa listede gösterir

In [ ]:
np.mean(df[df["Age"] == 41]["BMI"]) #41 yaşındaki insanların BMI değerleri 

In [ ]:
df["BMI"] = df["BMI"].fillna(np.mean(df[df["Age"] == 41]["BMI"])) #Eğer değersiz BMI datası olsaydı 41 yaşındakilerin BMI değerlerinin ort ile dolduracaktık
df[df["BMI"].isnull()] #kontrol

In [ ]:
list1 = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
sns.heatmap(df[list1].corr(), annot = True, fmt = ".2f" ) #korelasyon
plt.show()

In [ ]:
a = sns.factorplot(x = "Pregnancies", y = "Outcome", data = df, kind = "bar", size = 6) #doğum sayılarının şeker hastalığına etkisini inceledik
a.set_ylabels("Diabetes Probability")
plt.show()

In [ ]:
a = sns.factorplot(x = "Age", y = "Outcome", data = df, kind = "bar", size = 12) #Yaşın şeker hastalığına etkisini inceledik
a.set_ylabels("Diabetes Probability")
plt.show()

In [ ]:
a = sns.FacetGrid(df, col = "Outcome") 
a.map(sns.distplot, "Glucose", bins = 5)
plt.show()
#Glukoz seviyesi 130 seviyesinden fazla ise şeker hastası olma ihtimali artıyor

In [ ]:
a = sns.FacetGrid(df, col = "Outcome", row = "Pregnancies", size = 6)
a.map(plt.hist, "Age", bins = 6)
a.add_legend()
plt.show() 
#Outcome - Pregnancies - Age 

In [ ]:
a = sns.FacetGrid(df, row = "Age", size = 2)
a.map(sns.pointplot, "BMI","Outcome","Glucose")
a.add_legend()
plt.show()

In [ ]:
df["Pregnancies"].head()

In [ ]:
sns.countplot(x = "Pregnancies", data = df)
plt.show()

In [ ]:
g = sns.factorplot(x = "Pregnancies", y = "Outcome", data = df, kind = "bar")
g.set_ylabels("Diabetes Probability")
plt.show()

In [ ]:
df = pd.get_dummies(df, columns = ["Pregnancies"])
df.head()

In [ ]:
#df = df.set_index("Glucose") #index değiştirdik
df.head()

In [ ]:
df["Age"].head(10)

In [ ]:
sns.countplot(x = "Age", data = df)
plt.xticks(rotation = 60)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
df_len = len(df)
df_len

In [ ]:
test = df[500:]
test.drop(labels = ["Outcome"], axis = 1, inplace = True)

In [ ]:
test.head()

In [ ]:
train = df[:df_len]
x_train = train.drop(labels = "Outcome", axis = 1)
y_train = train["Outcome"]
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.33, random_state = 42)
print("X_train:", len(x_train))
print("X_test:", len(x_test))
print("y_train:", len(y_train))
print("y_test:", len(y_test))
print("test:", len(test))

In [ ]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
acc_log_train = round(logreg.score(x_train, y_train)*100,2)
acc_log_test = round(logreg.score(x_test, y_test)*100,2)
print("Training Accuracy: %{}".format(acc_log_train))
print("Test Accuracy: %{}".format(acc_log_test))

In [ ]:
random_state = 42
classifier = [DecisionTreeClassifier(random_state = random_state),
             SVC(random_state = random_state),
             RandomForestClassifier(random_state = random_state),
             LogisticRegression(random_state = random_state),
             KNeighborsClassifier()]

dt_param_grid = {"min_samples_split" : range(10,500,20),
                "max_depth": range(1,20,2)}

svc_param_grid = {"kernel" : ["rbf"],
                 "gamma": [0.001, 0.01, 0.1, 1],
                 "C": [1,10,50,100,200,300,1000]}

rf_param_grid = {"max_features": [1,3,10],
                "min_samples_split":[2,3,10],
                "min_samples_leaf":[1,3,10],
                "bootstrap":[False],
                "n_estimators":[100,300],
                "criterion":["gini"]}

logreg_param_grid = {"C":np.logspace(-3,3,7),
                    "penalty": ["l1","l2"]}

knn_param_grid = {"n_neighbors": np.linspace(1,19,10, dtype = int).tolist(),
                 "weights": ["uniform","distance"],
                 "metric":["euclidean","manhattan"]}
classifier_param = [dt_param_grid,
                   svc_param_grid,
                   rf_param_grid,
                   logreg_param_grid,
                   knn_param_grid]

In [ ]:
cv_result = []
best_estimators = []
for i in range(len(classifier)):
    clf = GridSearchCV(classifier[i], param_grid=classifier_param[i], cv = StratifiedKFold(n_splits = 10), scoring = "accuracy", n_jobs = -1,verbose = 1)
    clf.fit(x_train,y_train)
    cv_result.append(clf.best_score_)
    best_estimators.append(clf.best_estimator_)
    print(cv_result[i])

In [ ]:
cv_results = pd.DataFrame({"Cross Validation Means":cv_result, "ML Models":["DecisionTreeClassifier", "SVM","RandomForestClassifier",
             "LogisticRegression",
             "KNeighborsClassifier"]})

g = sns.barplot("Cross Validation Means", "ML Models", data = cv_results)
g.set_xlabel("Mean Accuracy")
g.set_title("Cross Validation Scores")

In [ ]:
votingC = VotingClassifier(estimators = [("dt",best_estimators[0]),
                                        ("rfc",best_estimators[2]),
                                        ("lr",best_estimators[3])],
                                        voting = "soft", n_jobs = -1)
votingC = votingC.fit(x_train, y_train)
print(accuracy_score(votingC.predict(x_test),y_test))

In [ ]:
test_outcome = pd.Series(votingC.predict(test), name = "Outcome").astype(int)
results = test_outcome
results.to_csv("diabetes.csv", index = False)